# Transformer/Diffusion Generative AI Model

## Flux

**What is Flux?**

Flux is a new AI image generation model developed by Black Forest Labs. It represents a significant advancement in AI-generated art, utilizing a “hybrid architecture” that combines transformer and diffusion techniques, scaled up to 12 billion parameters.

The model offers state-of-the-art performance image generation with top of the line prompt following, visual quality, image detail and output diversity.

***Note: If you don't have a GPU with more than 24 GB of memory you will likley become very frustrated and there is NO guarantee this code will work.***

**References:**

+ https://medium.com/@drmarcosv/how-does-flux-work-the-new-image-generation-ai-that-rivals-midjourney-7f81f6f354da#:~:text=Flux%20is%20a%20new%20AI,up%20to%2012%20billion%20parameters.
+ https://huggingface.co/black-forest-labs/FLUX.1-dev
+ https://github.com/black-forest-labs/flux/issues/39

In [1]:
###########################################
#- Minimal imports to start
###########################################
try:
    import sys
    import subprocess
    import importlib.util
    import atexit
except ImportError as e:
    print("There was a problem importing the most basic libraries necessary for this code.")
    print(repr(e))
    raise SystemExit("Stop right there!")

###########################################
#- Final Exit Routine
###########################################
@atexit.register
def goodbye():
    print("GOODBYE")


        
libraries=["pillow", "torch", "diffusers", "transformers", "accelerate", "sentencepiece", "bitsandbytes"]    
print(f"Validating environment for the following pip packages: {libraries}")

#load environment for non-generative libraries
try:
    for library in libraries:
      if library == "Pillow":
        spec = importlib.util.find_spec("PIL")
      else:
        spec = importlib.util.find_spec(library)
      if spec is None:
        print("...installing library " + library)
        subprocess.run(["pip", "install" , library, "--quiet"])
      else:
        print("...library " + library + " already installed.")
except (subprocess.CalledProcessError, Exception) as e:
    print("Error: Failed to install required packages, your code might not run properly.")
    print(repr(e))

Validating environment for the following pip packages: ['pillow', 'torch', 'diffusers', 'transformers', 'accelerate', 'sentencepiece', 'bitsandbytes']
...installing library pillow
...library torch already installed.
...library diffusers already installed.
...library transformers already installed.
...library accelerate already installed.
...library sentencepiece already installed.
...library bitsandbytes already installed.


# Variables

In [1]:
ENCODING  ="utf-8"

# Imports and Libraries

In [2]:
import os
try:    
    import torch
except ImportError as ie:
    debug.msg_warning("Failed to Pytorch, this could seriously impact functionality, be forewarned.")
    debug.msg_warning(f"...{repr(ie)}")
    pass
except Exception as e:
    debug.msg_warning("Failed to Pytorch, this could seriously impact functionality, be forewarned.")
    debug.msg_warning(f"...{repr(e)}")
    pass


os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
os.environ['PYTHONIOENCODING']=ENCODING

#Set GPU access
THE_DEVICE=0
os.environ['CUDA_VISIBLE_DEVICES']=f"{THE_DEVICE}"
device = torch.device(f"cuda:{THE_DEVICE}") 


try:
    print(f"{'Torch version':<40}#: {str(torch.__version__):<20}")
    print(f"{'     GPUs available?':<40}#: {torch.cuda.is_available()}")
    print(f"{'     count':<40}#: {torch.cuda.device_count()}")
    print(f"{'     current':<40}#: {torch.cuda.current_device()}")
    print(f"{'      device':<40}#: {torch.cuda.get_device_name(0)}")
    #device = torch.device("cuda:1")  # Use the second GPU (index starts from 0)
except Exception as e:
    pass


Torch version                           #: 2.4.0+cu121         
     GPUs available?                    #: True
     count                              #: 1
     current                            #: 0
      device                            #: Tesla T4


# Authenticate with Hugging FAce

Required as part of the agreement.  Follow the directions of 

In [3]:
from huggingface_hub import login
import accelerate

# Method 1: Using the login() function
login() # This will prompt you for your Hugging Face token

# Method 2: Passing the token directly
token = os.environ['HUGGINGFACE_API_KEY_FOR_FLUX']
login(token=token)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /home/jupyter/.cache/huggingface/token
Login successful


# Load Libraries in direct support of Flux and establish a "pipe"

A pipeline in Hugging Face is a function that takes raw text as input and processes it in a sequence of computing elements. It's a way to use models for inference, and it's designed to be easy to use even if you don't have experience with the underlying code. 

Here are some things a pipeline can do: 

+ Tokenize: Break the text into words or sub-words 
+ Feed to a model: Pass the tokenized text to a model that performs a specific task, such as entity recognition or text summarization 
+ Abstract complex code: Hide most of the complex code from the library with a simple API 
+ Work with multiple tasks: Perform tasks such as named entity recognition, masked language modeling, sentiment analysis, feature extraction, and question answering 
+ Work with multiple modalities: Use a pipeline for audio, vision, and multimodal tasks 

**Reference:**

+ https://huggingface.co/docs/transformers/main_classes/pipelines

In [4]:
#import libraries and local model into your system using Hugging Face's transformers.
from diffusers import FluxPipeline
from diffusers import DiffusionPipeline
import accelerate
 

#DEV version is more advanced but takes more resources.
#pipe = FluxPipeline.from_pretrained("black-forest-labs/FLUX.1-dev", torch_dtype=torch.bfloat16)

#Schnell is more streamlined
pipe = FluxPipeline.from_pretrained("black-forest-labs/FLUX.1-schnell")


#pipe.enable_model_cpu_offload() #save some VRAM by offloading the model to CPU. Remove this if you have enough GPU power
pipe.enable_sequential_cpu_offload()

'\n#import libraries and local model into your system using Hugging Face\'s transformers.\nfrom diffusers import FluxPipeline\nfrom diffusers import DiffusionPipeline\nimport accelerate\n \n#pipe = FluxPipeline.from_pretrained("black-forest-labs/FLUX.1-dev", torch_dtype=torch.bfloat16)\npipe = FluxPipeline.from_pretrained("black-forest-labs/FLUX.1-schnell")\n\n\n#Reference: https://github.com/black-forest-labs/flux/issues/39\n#pipe.enable_model_cpu_offload() #save some VRAM by offloading the model to CPU. Remove this if you have enough GPU power\npipe.enable_sequential_cpu_offload()\n'

# Invoke the model through the pipeline

Expect 2-5 minutes for output generation.

In [ ]:
#invoke the model and generate the output saving the file and showing the image
image = pipe(
    prompt,
    height=1024,
    width=1024,
    guidance_scale=3.5,
    num_inference_steps=50,
    max_sequence_length=512,
    #generator=torch.Generator("cpu").manual_seed(0)
).images[0]
image.save("flux-dev.png")
image.show()

'\n#invoke the model and generate the output saving the file and showing the image\nimage = pipe(\n    prompt,\n    height=1024,\n    width=1024,\n    guidance_scale=3.5,\n    num_inference_steps=50,\n    max_sequence_length=512,\n    #generator=torch.Generator("cpu").manual_seed(0)\n).images[0]\nimage.save("flux-dev.png")\nimage.show()\n'